# Métricas de Evaluación e Introducción a Scikit Learn
Hasta el momento, hemos implementado los algoritmos directamente. En la vida real, implementar los algoritmos desde cero no es la mejor idea. Scikit Learn es una librería de computación científica con clases que ya implementan algoritmos de aprendizaje como regresión lineal, logística, árboles de decisiones y Support Vector Machines (SVM).

En este cuaderno aprenderemos un poco de las herramientas que tenemos y aprenderemos a comparar diferentes algoritmos en el mismo problema.

Antes de comenzar, recuerda instalar scikit-learn desde tu ambiente con ```conda install scikit-learn```

In [ ]:
import pandas
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets # sklearn ya tiene varios datasets cargados
from sklearn import metrics # también tiene métricas para evaluar un modelo
from sklearn.linear_model import LinearRegression # Y ya tiene regresión lineal implementada!

In [ ]:
# Esta función grafica un clasificador
def plot_model(X, y, clf):
    plt.scatter(X[np.argwhere(y==0).flatten(),0],X[np.argwhere(y==0).flatten(),1],s = 50, color = 'blue', edgecolor = 'k')
    plt.scatter(X[np.argwhere(y==1).flatten(),0],X[np.argwhere(y==1).flatten(),1],s = 50, color = 'red', edgecolor = 'k')

    plt.xlim(-2.05,2.05)
    plt.ylim(-2.05,2.05)
    plt.grid(False)
    plt.tick_params(
        axis='x',
        which='both',
        bottom='off',
        top='off')

    r = np.linspace(-2.1,2.1,300)
    s,t = np.meshgrid(r,r)
    s = np.reshape(s,(np.size(s),1))
    t = np.reshape(t,(np.size(t),1))
    h = np.concatenate((s,t),1)

    z = clf.predict(h)

    s = s.reshape((np.size(r),np.size(r)))
    t = t.reshape((np.size(r),np.size(r)))
    z = z.reshape((np.size(r),np.size(r)))

    plt.contourf(s,t,z,colors = ['blue','red'],alpha = 0.2,levels = range(-1,2))
    if len(np.unique(z)) > 1:
        plt.contour(s,t,z,colors = 'k', linewidths = 2)
    plt.show()

## Regresión Lineal con muchas variables

In [ ]:
# Cargamos un dataset público de los precios de las casas de boston
housing_data = datasets.load_boston()
print("Features: " , housing_data.feature_names)
print("Ejemplo de un input: ", housing_data.data[0])
print("Ejemplo de un output: " , housing_data.target[0])

In [ ]:
# Creamos el modelo (no entrenado)
linear_regression_model = LinearRegression()

In [ ]:
# Hacemos un fit de la línea dado sus valores en x y en y.
linear_regression_model.fit(housing_data.data, housing_data.target)

In [ ]:
# Creamos predicciones de nuestro data. Nota que aquí estamos usando lo mismo con lo que entrenamos! (mala práctica)
predictions = linear_regression_model.predict(housing_data.data)

In [ ]:
# Evaluamos
error = metrics.mean_absolute_error(housing_data.target, predictions)
error

In [ ]:
# Evaluamos
error = metrics.mean_squared_error(housing_data.target, predictions)
error

In [ ]:
score = metrics.r2_score(housing_data.target, predictions)
score

## Implementación de Clasificación
Ahora veremos 3 diferentes técnicas para clasificar y evaluaremos cuál funciona mejor en varios datasets.

### Problema 1.
Iniciamos cargando la información usando pandas

In [ ]:
data = pandas.read_csv("data/small_test.csv")
data[:10]

En la siguiente celda graficamos por categoría. La respuesta de cómo hacer esto está en stackoverflow:
https://stackoverflow.com/questions/21654635/scatter-plots-in-pandas-pyplot-how-to-plot-by-category

In [ ]:
def plot_class(data):
    groups = data.groupby('y')
    fig, ax = plt.subplots()
    ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
    for name, group in groups:
        ax.plot(group.x1, group.x2, marker='o', linestyle='', ms=12, label=name)
    ax.legend()

    plt.show()
plot_class(data)

In [ ]:
# Separamos features de label
X = np.array(data[['x1', 'x2']])
y = np.array(data['y'])

In [ ]:
# Ejemplo de input
X[:10]

En la siguiente celda implementamos regresión logística. Son 5 pasos:
1. Cargamos el modelo de la librería donde está
2. Creamos una instancia de la técnica (LogisticRegression())
3. Hacemos un fit de X y y.
4. Predecimos con el clasificador para X.
5. Utilizamos accuracy_score para medir su desempeño

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X,y)
predictions = classifier.predict(X)
metrics.accuracy_score(y, predictions)
plot_model(X, y, classifier)

**TODO** Repite este proceso en las dos siguientes celdas con un árbol de decisión y con un SVC. 

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.tree import DecisionTreeClassifier
# Pon tu código
plot_model(X, y, classifier)

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
from sklearn.svm import SVC
# Pon tu código
plot_model(X, y, classifier)

### Problema 2.
En el anterior problema, estaba muy claras las separaciones. Ahora trabajaremos con información más compleja.

In [ ]:
data = pandas.read_csv("data/second_test.csv")
data[:10]

In [ ]:
plot_class(data)

In [ ]:
X = np.array(data[['x1', 'x2']])
y = np.array(data['y'])

Para este problema, vamos a utilizar un SVM (SVC). En el anterior ejercicio, no especificamos los parámetros del modelo. Ahora, vamos a configurar el SVC y ver cómo funciona mejor. Los valores más comunes son:
* kernel (string): linear, poly, rbf
* degree (int): grado del polinomio si elegiste poly
* gamma (float): Parámetro gama que se usa con kernel rbf.

classifier = SVC(kernel = ..., degree = ..., gamma = ...)

Prueba con linear primero, y luego prueba con otro. ¿Puedes lograr un accuracy de 1?

In [ ]:
classifier = SVC(kernel = 'linear')
classifier.fit(X,y)
predictions = classifier.predict(X)
metrics.accuracy_score(y, predictions)

In [ ]:
plot_model(X, y, classifier)

**Pregunta** Investiga en la documentación de SVC qué significa cada parámetro (kernel y gamma). ¿Cuál fue tu accuracy con las diferentes combinaciones de parámetros. ¿Por qué uno funcionó mejor que otro?

### Problema 3.
El objetivo de este problema es mostrar cómo hacer un split de información con scikit learn.

In [ ]:
data = pandas.read_csv("data/third_test.csv")
data[:10]

In [ ]:
plot_class(data)

In [ ]:
from sklearn.cross_validation import train_test_split

X = np.array(data[['x1', 'x2']])
y = np.array(data['y'])

# Usamos train_test_split para separar el 20% a testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2) 
y_test

In [ ]:
classifier = SVC(kernel = 'rbf', gamma = 5)

# Entrenamos con training data
classifier.fit(X_train, y_train)

# Evaluamos con testing data
predictions = classifier.predict(X_test)
metrics.accuracy_score(y_test, predictions)

In [ ]:
plot_model(X, y, classifier)

También podemos hacer K-Fold

In [ ]:
from sklearn.model_selection import KFold
X = np.array(data[['x1', 'x2']])
y = np.array(data['y'])
kf = KFold(n_splits=2)
kf.get_n_splits(X)

In [ ]:
print(kf)

In [ ]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

### Problema 4. 
Ahora veremos cómo aplicar Grid Search. Recuerda, a veces hay muchos parámetros para los modelos y Grid Search permite explorar eso.

In [ ]:
# Cargamos datos
data = pandas.read_csv("data/problem4.csv")
data[:5]

In [ ]:
plot_class(data)

In [ ]:
X = np.array(data[['x1', 'x2']])
y = np.array(data['y'])

In [ ]:
# Usamos train_test_split para separar el 20% a testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2) 
y_test

In [ ]:
# Creamos el clasificador
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

# Predecimos
train_predictions = classifier.predict(X_train)
test_predictions = classifier.predict(X_test)

plot_model(X, y, classifier)

In [ ]:
# Calculamos F1
from sklearn.metrics import f1_score
print('The Training F1 Score is', f1_score(train_predictions, y_train))
print('The Testing F1 Score is', f1_score(test_predictions, y_test))

Un F1 de 1 en el entrenamiento es una clara muestra de overfitting! Veamos cómo usar GridSearch

In [ ]:
from sklearn.metrics import make_scorer

# Creamos un clasificador
classifier = DecisionTreeClassifier()

# Hacemos lista de los parámetros que queremos probar
parameters = {'max_depth':[2,4,6,8,10],'min_samples_leaf':[2,4,6,8,10], 'min_samples_split':[2,4,6,8,10]}

# Hacemos un scorer para que evalue
scorer = make_scorer(f1_score)

# Creamos el Grid Object. 
grid_obj = GridSearchCV(classifier, parameters, scoring=scorer)

# Entrenamos
grid_fit = grid_obj.fit(X_train, y_train)

# Conseguimos el mejor estimador
best_clf = grid_fit.best_estimator_

# Ahora lo entrenamos en entrenamiento
best_clf.fit(X_train, y_train)

# Hacemos predicciones
best_train_predictions = best_clf.predict(X_train)
best_test_predictions = best_clf.predict(X_test)

# Calculamos f1
print('The training F1 Score is', f1_score(best_train_predictions, y_train))
print('The testing F1 Score is', f1_score(best_test_predictions, y_test))

# Graficamos
plot_model(X, y, best_clf)

# Veamos los parámetros del mejor modelo
best_clf

In [ ]:
# Creamos un scorer
scorer = make_scorer(f1_score)

In [ ]:
# Importamos GridSearchCV
from sklearn.model_selection import GridSearchCV

# Seleccionamos parámetros
parameters = {'kernel':['poly', 'rbf'],'C':[0.1, 1, 10]}

### Problema 5. 
En este problema, haz los siguientes pasos

1. Grafica
2. Carga la información de x y la información de y
3. Separa en training y testing.
4. Crea un clasificador SVC, pero no lo entrenes.
5. Utiliza GridSearch para encontrar el mejor model.
6. Con el mejor modelo, vuelve a entrenarlo con el training. 
7. Calcula el F1. Revisa http://scikit-learn.org/stable/modules/classes.html#classification-metrics y elige otras dos maneras de calcular el score. 
8. Grafica el clasificador.

In [ ]:
data = pandas.read_csv("data/small_test.csv")